In [37]:
import requests as req 
from bs4 import BeautifulSoup   
import time
import json
import tqdm
import numpy as np
import pandas as pd

websites = {'TASS' : ['https://tass.ru/rss/v2.xml', ''],
            'LENTA.RU' : ['https://lenta.ru/rss', ''],
            'ВЕДОМОСТИ' : ['https://www.vedomosti.ru/rss/news', '']
           }

data = {"data" : []  }
all_data = {"data" : []  }
df = pd.DataFrame(data['data'])
df_all = pd.DataFrame(all_data['data'])


def parser(df, df_all):        
        
    for site in range(len(websites)):
        
        exitFlag=False
        
        data = {"data" : [] }
        all_data = {"data" : [] }
       
        url = list(websites.values())[site][0]
                
        resp = req.get(url)
        soup  = BeautifulSoup(resp.content, features="lxml")

        tag = soup.find_all('item')
        
        for item in tag:
            
            title = item.find('title')
            
            if title.text != list(websites.values())[site][1]: 
                exitFlag=True 
                
                category = item.find('category')

                date = item.find('pubdate')

                data['data'].append({"title" : title.text, "category": category.text,
                                     "pubdate" : date.text, "site" : list(websites.keys())[site]
                                    })
                all_data['data'].append({"site" : list(websites.keys())[site], "item" : item.text})


                with open(f"all_data_{pd.Timestamp('today'):%d-%m-%Y-%H-%M}.json", "w", encoding="utf-8") as file:
                      json.dump(all_data, file, ensure_ascii = False)
                        
            else:
                break                                
                
        if(exitFlag):
            df = pd.concat([pd.DataFrame(data['data']), df], ignore_index=True)
            df_all = pd.concat([pd.DataFrame(all_data['data']), df_all], ignore_index=True)        
            websites[list(websites.keys())[site]][1] = df.iloc[0]['title']   
          
    return df, df_all

df, df_all  = parser(df, df_all) 
df.sample(10)

/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup cons

,title,category,pubdate,site
399,Россиянин избивал сына-подростка и заставлял е...,Россия,"Fri, 31 Mar 2023 15:34:00 +0300",LENTA.RU
283,Страны ЕС призвали ЕК вмешаться в кризис с укр...,Мир,"Fri, 31 Mar 2023 19:27:00 +0300",LENTA.RU
289,Врач рассказала о борьбе с болезнями с помощью...,Забота о себе,"Fri, 31 Mar 2023 19:15:00 +0300",LENTA.RU
150,Собянин: работа столичных военкоматов не связа...,Общество,"Wed, 29 Mar 2023 22:00:47 +0300",ВЕДОМОСТИ
187,Глава МАГАТЭ прибыл на Запорожскую АЭС,Политика,"Wed, 29 Mar 2023 13:53:47 +0300",ВЕДОМОСТИ
268,На Украине сообщили об очередях в аптеки из-за...,Бывший СССР,"Fri, 31 Mar 2023 20:00:00 +0300",LENTA.RU
418,Bloomberg: оппозиция Венесуэлы лишилась доступ...,Международная панорама,"Fri, 31 Mar 2023 23:52:32 +0300",TASS
198,Прокуратура запросила пожизненный срок для нап...,Общество,"Wed, 29 Mar 2023 12:52:43 +0300",ВЕДОМОСТИ
424,Болгарская полиция задержала подозреваемого в ...,Происшествия,"Fri, 31 Mar 2023 23:28:23 +0300",TASS
113,«Медвестник»: число программ суррогатного мате...,Общество,"Thu, 30 Mar 2023 14:50:24 +0300",ВЕДОМОСТИ


In [38]:
if len(df['title'].unique()) < len(df):
    print('There are duplicates in the data')
    df = df.drop_duplicates('title', keep='first')

There are duplicates in the data


In [39]:
if df['title'].isnull().any():
    print('There are missing values in this dataset')
    df = df[df['title'].isnull()==False]

In [40]:
df.head()

,title,category,pubdate,site
0,Орбан оценил угрозу начала третьей мировой войны,Политика,"Fri, 31 Mar 2023 22:32:36 +0300",ВЕДОМОСТИ
1,Мишустин подписал распоряжение о создании ремо...,Политика,"Fri, 31 Mar 2023 22:07:26 +0300",ВЕДОМОСТИ
2,Ростовский губернатор сообщил об устранении по...,Общество,"Fri, 31 Mar 2023 21:45:09 +0300",ВЕДОМОСТИ
3,Минюст включил в реестр иноагентов музыканта М...,Общество,"Fri, 31 Mar 2023 21:23:15 +0300",ВЕДОМОСТИ
4,Руководство МФТИ отказалось от обязательного и...,Общество,"Fri, 31 Mar 2023 21:01:51 +0300",ВЕДОМОСТИ


3) РАБОТАЕМ С КАТЕГОРИЯМИ

- Общее количество новостей из всех источников по данной категории за все время

In [41]:
df_1 = pd.DataFrame(df['category'].value_counts())
df_1

,category
Политика,83
Общество,71
Мир,32
Россия,31
Экономика,30
Бывший СССР,24
Международная панорама,22
Спорт,21
Бизнес,21
Экономика и бизнес,18


- Количество новостей данной категории для каждого из источников за все время

In [42]:
new_df  = df.iloc[:, [0,1,3]]
new_df_1 = pd.DataFrame(new_df.groupby('site')['category'].value_counts())
new_df_1.rename(columns={'category': 'Count'}, inplace=True)

new_df_1

Count
site      category                     
LENTA.RU  Мир                        32
          Россия                     31
          Бывший СССР                24
          Силовые структуры          16
          Экономика                  14
          Среда обитания             13
          Интернет и СМИ             10
          Спорт                      10
          Ценности                   10
          Из жизни                    9
          Забота о себе               8
          Наука и техника             7
          Культура                    6
          Путешествия                 5
          Моя страна                  3
TASS      Международная панорама     22
          Экономика и бизнес         18
          Общество                   14
          Спорт                      11
          Политика                    8
          Происшествия                8
          Культура                    6
          Москва                      6
          Армия и ОПК                 4
          В стране                    3
ВЕДОМОСТИ Политика                   75
          Общество                   57
          Бизнес                     21
          Финансы                    18
          Экономика                  16
          Технологии                  6
          Авто                        3
          Медиа                       3
          Недвижимость                1

- Общее количество новостей из всех источников по данной категории за последние сутки

In [43]:
df_news = df.copy()
df_news['day_of_week'] = 1
df_news['date'] = 1
df_news

,title,category,pubdate,site,day_of_week,date
0,Орбан оценил угрозу начала третьей мировой войны,Политика,"Fri, 31 Mar 2023 22:32:36 +0300",ВЕДОМОСТИ,1,1
1,Мишустин подписал распоряжение о создании ремо...,Политика,"Fri, 31 Mar 2023 22:07:26 +0300",ВЕДОМОСТИ,1,1
2,Ростовский губернатор сообщил об устранении по...,Общество,"Fri, 31 Mar 2023 21:45:09 +0300",ВЕДОМОСТИ,1,1
3,Минюст включил в реестр иноагентов музыканта М...,Общество,"Fri, 31 Mar 2023 21:23:15 +0300",ВЕДОМОСТИ,1,1
4,Руководство МФТИ отказалось от обязательного и...,Общество,"Fri, 31 Mar 2023 21:01:51 +0300",ВЕДОМОСТИ,1,1
...,...,...,...,...,...,...
495,"Минюст США подал иск против компании, чей поез...",Общество,"Fri, 31 Mar 2023 20:36:59 +0300",TASS,1,1
496,"""Ростелеком"" получил косвенный контроль в пров...",Экономика и бизнес,"Fri, 31 Mar 2023 20:35:07 +0300",TASS,1,1
497,Постпред Белоруссии при ООН предложил Западу р...,Международная панорама,"Fri, 31 Mar 2023 20:32:22 +0300",TASS,1,1
498,"Эксперт считает, что вступление Финляндии в НА...",Политика,"Fri, 31 Mar 2023 20:32:10 +0300",TASS,1,1


In [44]:
for index, row in enumerate(df_news['pubdate'].str.split(',')):
    #print(row)
    df_news['day_of_week'][index] = row[0]
    df_news['date'][index] = " ".join(row[1].split()[:-1])
    #df_news.sort(key=lambda date: datetime.strptime(date, "%m-%Y"))
 

/var/folders/vh/_yg98lps1dvdkkx_jvfjzsm80000gp/T/ipykernel_1813/3441872984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['day_of_week'][index] = row[0]
/var/folders/vh/_yg98lps1dvdkkx_jvfjzsm80000gp/T/ipykernel_1813/3441872984.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['date'][index] = " ".join(row[1].split()[:-1])


In [ ]:
df_news.sort_values(['date'], ascending = False)
df_news.sample(7)

In [14]:
conda install psycopg2

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - psycopg2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    psycopg2-2.9.3             |   py39h0a4fc7d_0         143 KB
    ------------------------------------------------------------
                                           Total:         143 KB

The following NEW packages will be INSTALLED:

  psycopg2           pkgs/main/osx-64::psycopg2-2.9.3-py39h0a4fc7d_0 None



psycopg2-2.9.3       | 143 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: | WARNING conda.core.path_actions:verify(962): Unable to create environments file. Path not 

In [48]:
import psycopg2

conn = psycopg2.connect(
    database="final project",
    user="postgres",
    password="localhost",
    host='127.0.0.1', 
    port='5432'
)

cur = conn.cursor()

In [55]:
# Creating table as per requirement
sql = """
CREATE TABLE IF NOT EXISTS CATEGORIES (
    ID INT PRIMARY KEY,
    category CHARACTER VARYING(100) NOT NULL 
)
"""

conn.autocommit = True
cur.execute(sql)


In [56]:
sql2 = """
CREATE TABLE IF NOT EXISTS SITES (
    ID INT PRIMARY KEY,
    SITE CHARACTER VARYING(80) NOT NULL 
)
"""

conn.autocommit = True
cur.execute(sql2)

In [69]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS NEWS (
    ID INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY, 
    date TIMESTAMP NOT NULL, 
    date_of_week CHARACTER VARYING(30) NOT NULL,
    category VARCHAR(100) NOT NULL

    )
"""

conn.autocommit = True
cur.execute(create_posts_table)

In [76]:
df_news.sample(7)

,title,category,pubdate,site,day_of_week,date
94,В МИД РФ сообщили о выдаче виз иностранным жур...,Общество,"Thu, 30 Mar 2023 17:54:15 +0300",ВЕДОМОСТИ,Thu,30 Mar 2023 17:54:15
345,Продюсер популярного шоу рассказал о «психолог...,Интернет и СМИ,"Fri, 31 Mar 2023 17:19:00 +0300",LENTA.RU,Fri,31 Mar 2023 17:11:43
120,Песков рассказал о возможности эвакуации монах...,Политика,"Thu, 30 Mar 2023 13:44:34 +0300",ВЕДОМОСТИ,Thu,30 Mar 2023 13:44:34
199,Генпрокуратуру попросили проверить Меладзе на ...,Политика,"Wed, 29 Mar 2023 12:43:20 +0300",ВЕДОМОСТИ,Wed,29 Mar 2023 12:43:20
463,Московский зоопарк выясняет обстоятельства отр...,Москва,"Fri, 31 Mar 2023 21:41:04 +0300",TASS,Fri,31 Mar 2023 21:39:21
72,Япония ввела запрет на экспорт алюминия и стал...,Политика,"Fri, 31 Mar 2023 07:50:07 +0300",ВЕДОМОСТИ,Fri,31 Mar 2023 07:50:07
437,Около 91 тыс. семей участников СВО находятся п...,Общество,"Fri, 31 Mar 2023 22:41:55 +0300",TASS,Fri,31 Mar 2023 22:40:11


#Икрементальный режим